# Day 23
https://adventofcode.com/2018/day/23

In [1]:
import aocd
data = aocd.get_data(year=2018, day=23)

In [2]:
from dataclasses import dataclass
import regex as re
import z3

In [3]:
re_nanobot = re.compile(r'pos=<([-\d]+),([-\d]+),([-\d]+)>, r=(\d+)')

In [4]:
@dataclass(frozen=True, order=True)
class Point():
    x: int
    y: int
    z: int
    
    def __add__(self, other):
        return Point(self.x + other.x, self.y + other.y, self.z + other.z)
    
    def distance_from(self, other):
        return abs(self.x - other.x) + abs(self.y - other.y) + abs(self.z - other.z)

In [5]:
def zabs(value):
    return z3.If(value >= 0, value, -value)

In [6]:
@dataclass(frozen=True, order=True)
class Nanobot():
    radius: int
    location: Point
    
    @classmethod
    def from_regex_groups(cls, groups):
        x, y, z, r = [int(val) for val in groups]
        return cls(r, Point(x, y, z))
    
    def in_range_of(self, location):
        return self.location.distance_from(location) <= self.radius
    
    def z3_in_range_of(self, x, y, z):
        nx, ny, nz = self.location.x, self.location.y, self.location.z
        return z3.If(zabs(x - nx) + zabs(y - ny) + zabs(z - nz) <= self.radius, 1, 0)
    
    def number_in_range(self, nanobots):
        return sum(1 for nanobot in nanobots if self.in_range_of(nanobot))

In [7]:
def read_nanobots(text):
    return {Nanobot.from_regex_groups(groups) for groups in re_nanobot.findall(text)}

In [8]:
def in_range_of_strongest(nanobots):
    strongest = max(nanobot for nanobot in nanobots)
    return strongest.number_in_range(nanobot.location for nanobot in nanobots)

In [9]:
def zabs(value):
    return z3.If(value > 0, value, -value)

In [10]:
def distance_to_point_in_range_of_most_nanobots(nanobots):
    x = z3.Int('x')
    y = z3.Int('y')
    z = z3.Int('z')
    
    bots_in_range = {
        z3.Int(f'in_range_{botno}'): bot.z3_in_range_of(x, y, z)
        for botno, bot in enumerate(nanobots)
    }
    
    opt = z3.Optimize()
    for variable, constraint in bots_in_range.items():
        opt.add(variable == constraint)
    
    bot_count = z3.Int('bot_count')
    opt.add(bot_count == sum(bots_in_range.keys()))
    
    dist = z3.Int('dist')
    opt.add(dist == zabs(x) + zabs(y) + zabs(z))
    
    highest_bot_count = opt.maximize(bot_count)
    nearest_point = opt.minimize(dist)
    
    opt.check()

    return opt.lower(nearest_point)

In [11]:
nanobots = read_nanobots(data)
p1 = in_range_of_strongest(nanobots)
print(f'Part 1: {p1}')
p2 = distance_to_point_in_range_of_most_nanobots(nanobots)
print(f'Part 2: {p2}')

Part 1: 497
Part 2: 85761543
